In [1]:
import cv2
import numpy as np
import time

In [2]:
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly (mask, vertices, match_mask_color )
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [3]:
def draw_the_lines(img, lines):
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0,255,0), thickness=4)
        
    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

In [4]:
def process(image):
    height = image.shape[0]
    width = image.shape[1]
    region_of_interest_vertices = [
        (0, height),
        (width/2, height*3/4),
        (width, height)]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    canny_image = cv2.Canny(gray_image, 100, 200)
    cropped_image = region_of_interest(canny_image, np.array([region_of_interest_vertices], np.int32))
    
    lines = cv2.HoughLinesP(cropped_image,
                      rho = 2,
                      theta = np.pi/180,
                      threshold=50,
                      lines = np.array([]),
                      minLineLength=40,
                      maxLineGap=100)
    image_with_lines = draw_the_lines(image, lines)
    return image_with_lines

In [5]:
cap = cv2.VideoCapture('test.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if ret == False:
        break
    frame = process(frame)
    cv2.imshow('frame', frame)
    #time.sleep(0.034)  #每秒播29點多張，似waitkey
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()